#Homogeneity analysis of food establishments in Pachuca, Mexico using K-Means




The steps of the Project:


*   Scrap the data for the analysis
*   Cleaning and Wrangling of the Data
*   Fit a K-Means Model
*   Generate the figures for the Report

I will make aclarations if the code needs it.

In [ ]:
#Importing the Libraries
import json, requests, folium
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

Scrappe for the Data of the Locations

In [ ]:
url = 'https://codigo-postal.co/mexico/hidalgo/pachuca-de-soto/'

In [ ]:
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

dfs = pd.read_html(page.text)
hidalgo_array = np.array(dfs)[0]
hidalgo_array = pd.DataFrame(hidalgo_array[1:], columns=['Código Postal', 'Asentamiento',	'Municipio', 'Estado', 'Tipo'])

hidalgo_array.head()

,Código Postal,Asentamiento,Municipio,Estado,Tipo
0,42064,Fraccionamiento 16 de Enero,Pachuca de Soto,Hidalgo,Urbano
1,42092,Fraccionamiento 18 de Marzo,Pachuca de Soto,Hidalgo,Urbano
2,42039,Colonia 20 de Noviembre,Pachuca de Soto,Hidalgo,Urbano
3,42075,Fraccionamiento 2a. Rinconada de Santiago,Pachuca de Soto,Hidalgo,Urbano
4,42039,Colonia Abundio de Antorcha Campesina,Pachuca de Soto,Hidalgo,Urbano


In [ ]:
hidalgo_array.to_csv('hidalgo_raw.csv')

The data for Mexican Cities are small, in our location only exist 23 postal codes [Link](https://es.wikipedia.org/wiki/Anexo:C%C3%B3digos_postales_mexicanos#Hidalgo)

The most importat for the state of Hidalgo are:

* 42083 Pachuca, Quinta Bonita
* 42080 Pachuca, Col. Centro Minero
* 43600 Tulancingo, Col. Centro
* 43200 Zacualtipán, Zacualtipán de Ángeles, Hgo.

But only the data for the county scrapped will be considered

In [ ]:
hidalgo_array = hidalgo_array.groupby('Código Postal').agg(lambda x : list(x)[0])

hidalgo_array.reset_index(inplace=True)

hidalgo_array

,Código Postal,Asentamiento,Municipio,Estado,Tipo
0,42000,Colonia Centro,Pachuca de Soto,Hidalgo,Urbano
1,42015,Barrio Asta Bandera,Pachuca de Soto,Hidalgo,Urbano
2,42027,Colonia Antonio del Castillo,Pachuca de Soto,Hidalgo,Urbano
3,42034,Colonia Aquiles Serdán,Pachuca de Soto,Hidalgo,Urbano
4,42037,Equipamiento Cereso de Pachuca,Pachuca de Soto,Hidalgo,Urbano
5,42039,Colonia 20 de Noviembre,Pachuca de Soto,Hidalgo,Urbano
6,42055,Colonia Anáhuac,Pachuca de Soto,Hidalgo,Urbano
7,42060,Colonia Arconada,Pachuca de Soto,Hidalgo,Urbano
8,42064,Fraccionamiento 16 de Enero,Pachuca de Soto,Hidalgo,Urbano
9,42070,Colonia Buenos Aires,Pachuca de Soto,Hidalgo,Urbano


In [ ]:
#install library for geolocation

!pip install --quiet pgeocode
import pgeocode

In [ ]:
#Fetch latitude and Longitude

data_hidalgo = hidalgo_array

data_hidalgo['Latitude'] = 0
data_hidalgo['Longitude'] = 0

for i in range(data_hidalgo.shape[0]):
  nomi = pgeocode.Nominatim('mx')
  postal_code = int(hidalgo_array['Código Postal'][i])
  geo_data = nomi.query_postal_code(postal_code)
  latitude = str(geo_data['latitude'])
  longitude = str(geo_data['longitude'])
  data_hidalgo['Latitude'][i] = latitude
  data_hidalgo['Longitude'][i] = longitude

data_hidalgo.columns = ['Postal Code','Neighborhood',	'County',	'State', 'County Type', 'Latitude','Longitude']

data_hidalgo.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Postal Code,Neighborhood,County,State,County Type,Latitude,Longitude
0,42000,Colonia Centro,Pachuca de Soto,Hidalgo,Urbano,20.1021,-98.7823
1,42015,Barrio Asta Bandera,Pachuca de Soto,Hidalgo,Urbano,20.1021,-98.76713333333332
2,42027,Colonia Antonio del Castillo,Pachuca de Soto,Hidalgo,Urbano,20.1283,-98.7391
3,42034,Colonia Aquiles Serdán,Pachuca de Soto,Hidalgo,Urbano,20.133,-98.7786
4,42037,Equipamiento Cereso de Pachuca,Pachuca de Soto,Hidalgo,Urbano,20.1229,-98.766


In [ ]:
data_hidalgo.to_csv('hidalgo_data.csv')

In [ ]:
#Function to get the categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#ApiCalls and create the DataFrame

url = 'https://api.foursquare.com/v2/venues/explore'

stores_dataframe = pd.DataFrame()

for i in range(data_hidalgo.shape[0]):
  lat = data_hidalgo['Latitude'].iloc[i]
  lon = data_hidalgo['Longitude'].iloc[i]
  coordinates = str(lat)+','+str(lon)

  params = dict(
      client_id='QSREYUFBGNHFPT01D550TW2EEUQJO252ZEOJZF3KXQYOYIYO',
      client_secret='DWAWJIBIJ4OMNV4ZQ3RAFHKDKAVVU0LQM3MSCXRF10VRD2EA',
      v='20180323',
      ll=coordinates,
      query='food',
      limit=20)
  
  resp = requests.get(url=url, params=params)
  data = json.loads(resp.text)
  items = data['response']['groups'][0]['items']
  dataframe = json_normalize(items)
  filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
  dataframe_filtered = dataframe.loc[:, filtered_columns]
  dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
  dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
  dataframe_filtered['Neighborhood'] = data_hidalgo['Neighborhood'].iloc[i]
  dataframe_filtered['Neighborhood Latitude'] = data_hidalgo['Latitude'].iloc[i]
  dataframe_filtered['Neighborhood Longitude'] = data_hidalgo['Longitude'].iloc[i]
  stores_dataframe = pd.concat([stores_dataframe,dataframe_filtered])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [ ]:
stores_dataframe.shape

(480, 19)

In [ ]:
stores_dataframe.head(6)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,neighborhood,postalCode,id,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,BIG APPLE,Bagel Shop,Plaza Gelka,Camino Real De La Plata 150,20.097525,-98.781234,"[{'label': 'display', 'lat': 20.09752526992193...",521,MX,Pachuca,Hidalgo,México,"[Plaza Gelka (Camino Real De La Plata 150), Pa...",NaN,NaN,56a406d9498ee1e859025eea,Colonia Centro,20.1021,-98.7823
1,Asados Don Abel,BBQ Joint,Blvd. Luis Donaldo Colosio,Ramón G. Bonfil,20.111201,-98.774273,"[{'label': 'display', 'lat': 20.11120133658232...",1315,MX,Pachuca,Hidalgo,México,"[Blvd. Luis Donaldo Colosio (Ramón G. Bonfil),...",NaN,NaN,51649680e4b069659ded94a9,Colonia Centro,20.1021,-98.7823
2,Rata Roja,Irish Pub,Camino Real de la Plata #135,NaN,20.096921,-98.771823,"[{'label': 'display', 'lat': 20.09692096964467...",1237,MX,Pachuca,Hidalgo,México,"[Camino Real de la Plata #135, Pachuca, Hidalg...",Zona Plateada,NaN,4bd23b0c9854d13a604ffb4d,Colonia Centro,20.1021,-98.7823
3,Puerto del Bife,Restaurant,"Camino Real la Plata, Lot. 15",Zona Plateada,20.094894,-98.770635,"[{'label': 'display', 'lat': 20.09489405941788...",1459,MX,Pachuca,Hidalgo,México,"[Camino Real la Plata, Lot. 15 (Zona Plateada)...",NaN,NaN,5450097e498e1e96446b2995,Colonia Centro,20.1021,-98.7823
4,Wingstop,Wings Joint,Camino Real de La Plata 70,NaN,20.096875,-98.777258,"[{'label': 'display', 'lat': 20.09687465536661...",784,MX,Pachuca,Hidalgo,México,"[Camino Real de La Plata 70, 42084 Pachuca, Hi...",NaN,42084,58ddb93b6f20d105826bc400,Colonia Centro,20.1021,-98.7823
5,Texas Ribs Via Platino,American Restaurant,NaN,NaN,20.098962,-98.768664,"[{'label': 'display', 'lat': 20.09896248237475...",1467,MX,Pachuca,Hidalgo,México,"[42084 Pachuca, Hidalgo, México]",NaN,42084,5aff5f3567e5f2002ce65815,Colonia Centro,20.1021,-98.7823


In [ ]:
print(stores_dataframe['categories'].unique())
print(len(stores_dataframe['categories'].unique()))

['Bagel Shop' 'BBQ Joint' 'Irish Pub' 'Restaurant' 'Wings Joint'
 'American Restaurant' 'Spanish Restaurant' 'Italian Restaurant'
 'Taco Place' 'Mexican Restaurant' 'Snack Place' 'Sushi Restaurant'
 'Sandwich Place' 'Seafood Restaurant' 'Burger Joint' 'Tex-Mex Restaurant'
 'Fast Food Restaurant' 'Deli / Bodega' 'Diner' 'Café'
 'Argentinian Restaurant' 'Pizza Place' 'Bakery' 'Food Truck'
 'Empanada Restaurant' 'Steakhouse' 'Japanese Restaurant' 'Breakfast Spot'
 'Gastropub' 'Vegetarian / Vegan Restaurant' 'Food' 'Buffet'
 'Asian Restaurant' 'Brazilian Restaurant' 'Fried Chicken Joint']
35


In spanish, the categories Coffee Shop, Cafe and Cafeteria are the equivalent, and for "food", "Mexican Restaurant" and "Restaurant" is much the same

So, We gonna correct that

In [ ]:
stores_dataframe.reset_index(inplace=True)
stores_dataframe.drop('index', axis=1)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,neighborhood,postalCode,id,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,BIG APPLE,Bagel Shop,Plaza Gelka,Camino Real De La Plata 150,20.097525,-98.781234,"[{'label': 'display', 'lat': 20.09752526992193...",521,MX,Pachuca,Hidalgo,México,"[Plaza Gelka (Camino Real De La Plata 150), Pa...",NaN,NaN,56a406d9498ee1e859025eea,Colonia Centro,20.1021,-98.7823
1,Asados Don Abel,BBQ Joint,Blvd. Luis Donaldo Colosio,Ramón G. Bonfil,20.111201,-98.774273,"[{'label': 'display', 'lat': 20.11120133658232...",1315,MX,Pachuca,Hidalgo,México,"[Blvd. Luis Donaldo Colosio (Ramón G. Bonfil),...",NaN,NaN,51649680e4b069659ded94a9,Colonia Centro,20.1021,-98.7823
2,Rata Roja,Irish Pub,Camino Real de la Plata #135,NaN,20.096921,-98.771823,"[{'label': 'display', 'lat': 20.09692096964467...",1237,MX,Pachuca,Hidalgo,México,"[Camino Real de la Plata #135, Pachuca, Hidalg...",Zona Plateada,NaN,4bd23b0c9854d13a604ffb4d,Colonia Centro,20.1021,-98.7823
3,Puerto del Bife,Restaurant,"Camino Real la Plata, Lot. 15",Zona Plateada,20.094894,-98.770635,"[{'label': 'display', 'lat': 20.09489405941788...",1459,MX,Pachuca,Hidalgo,México,"[Camino Real la Plata, Lot. 15 (Zona Plateada)...",NaN,NaN,5450097e498e1e96446b2995,Colonia Centro,20.1021,-98.7823
4,Wingstop,Wings Joint,Camino Real de La Plata 70,NaN,20.096875,-98.777258,"[{'label': 'display', 'lat': 20.09687465536661...",784,MX,Pachuca,Hidalgo,México,"[Camino Real de La Plata 70, 42084 Pachuca, Hi...",NaN,42084,58ddb93b6f20d105826bc400,Colonia Centro,20.1021,-98.7823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,"Taquería ""Chompomóvil""",Food Truck,"Av. Ferrocarril Hidalgo, Col. Villas de Pachuca",Av. Cisnes Norte,20.055433,-98.779526,"[{'label': 'display', 'lat': 20.05543283247722...",2943,MX,Pachuca,Hidalgo,México,"[Av. Ferrocarril Hidalgo, Col. Villas de Pachu...",NaN,42083,4f472b50108184d2b3620053,Colonia Caminera,20.0342,-98.7963
476,Don Guajolote,Fast Food Restaurant,NaN,casi esq. Germán Corona del Rosal,20.069286,-98.775914,"[{'label': 'display', 'lat': 20.06928560582272...",4449,MX,Pachuca,Hidalgo,México,"[Pachuca, Hidalgo, México]",C. Doria,NaN,4f77d3d4e4b00562b63d8928,Colonia Caminera,20.0342,-98.7963
477,Benjie's,Burger Joint,Av. Magisterio #86 Col. Tulipanes,NaN,20.060128,-98.772591,"[{'label': 'display', 'lat': 20.06012778373483...",3804,MX,Mineral de la Reforma,Hidalgo,México,"[Av. Magisterio #86 Col. Tulipanes, 42185 Min...",NaN,42185,5896299018dc5328f394829b,Colonia Caminera,20.0342,-98.7963
478,"Tacos de Guisado ""El Güero""",Taco Place,Av San Marcos s/n,Blvd Nuevo Hidalgo,20.065772,-98.770073,"[{'label': 'display', 'lat': 20.06577212003633...",4458,MX,Mineral de la Reforma,Hidalgo,México,"[Av San Marcos s/n (Blvd Nuevo Hidalgo), Miner...",NaN,NaN,4f58f135e4b059140e8a2c5f,Colonia Caminera,20.0342,-98.7963


In [ ]:
list_1 = list(stores_dataframe[stores_dataframe['categories'] == 'Café'].index.values)
list_2 = list(stores_dataframe[stores_dataframe['categories'] == 'Coffee Shop'].index.values)
list_3 = list(stores_dataframe[stores_dataframe['categories'] == 'Cafeteria'].index.values)

columns_list = list_1 + list_2 + list_3
columns_list

stores_dataframe['categories'].iloc[columns_list] = 'Coffee'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
list_1 = list(stores_dataframe[stores_dataframe['categories'] == 'Mexican Restaurant'].index.values)
list_2 = list(stores_dataframe[stores_dataframe['categories'] == 'Restaurant'].index.values)
list_3 = list(stores_dataframe[stores_dataframe['categories'] == 'Food'].index.values)

columns_list = list_1 + list_2 + list_3
columns_list

stores_dataframe['categories'].iloc[columns_list] = 'Restaurant'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#Sanity check
print(stores_dataframe['categories'].unique())
print(len(stores_dataframe['categories'].unique()))

['Bagel Shop' 'BBQ Joint' 'Irish Pub' 'Restaurant' 'Wings Joint'
 'American Restaurant' 'Spanish Restaurant' 'Italian Restaurant'
 'Taco Place' 'Snack Place' 'Sushi Restaurant' 'Sandwich Place'
 'Seafood Restaurant' 'Burger Joint' 'Tex-Mex Restaurant'
 'Fast Food Restaurant' 'Deli / Bodega' 'Diner' 'Coffee'
 'Argentinian Restaurant' 'Pizza Place' 'Bakery' 'Food Truck'
 'Empanada Restaurant' 'Steakhouse' 'Japanese Restaurant' 'Breakfast Spot'
 'Gastropub' 'Vegetarian / Vegan Restaurant' 'Buffet' 'Asian Restaurant'
 'Brazilian Restaurant' 'Fried Chicken Joint']
33


In [ ]:
stores_dataframe = stores_dataframe[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','name','lat','lng','categories']]
stores_dataframe.columns = ['Neighborhood',	'Neighborhood Latitude','Neighborhood Longitude',	'Venue',	'Venue Latitude',	'Venue Longitude',	'Venue Category']
stores_dataframe.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Colonia Centro,20.1021,-98.7823,BIG APPLE,20.097525,-98.781234,Bagel Shop
1,Colonia Centro,20.1021,-98.7823,Asados Don Abel,20.111201,-98.774273,BBQ Joint
2,Colonia Centro,20.1021,-98.7823,Rata Roja,20.096921,-98.771823,Irish Pub
3,Colonia Centro,20.1021,-98.7823,Puerto del Bife,20.094894,-98.770635,Restaurant
4,Colonia Centro,20.1021,-98.7823,Wingstop,20.096875,-98.777258,Wings Joint


In [ ]:
stores_dataframe.to_csv('stores_df.csv')

The classic one-hot encoding

In [ ]:
#One-hot encoding for K-Means Clustering

one_hot_df = pd.get_dummies(stores_dataframe[['Venue Category']], prefix="", prefix_sep="")

one_hot_df['Neighborhood'] = stores_dataframe['Neighborhood']
columns = list(one_hot_df.columns)
columns.remove('Neighborhood')
columns = ['Neighborhood'] + columns
one_hot_df = one_hot_df[columns]

one_hot_df.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Coffee,Deli / Bodega,Diner,Empanada Restaurant,Fast Food Restaurant,Food Truck,Fried Chicken Joint,Gastropub,Irish Pub,Italian Restaurant,Japanese Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Wings Joint
0,Colonia Centro,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Colonia Centro,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Colonia Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Colonia Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Colonia Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
hidalgo_grouped = one_hot_df.groupby('Neighborhood').mean().reset_index()
hidalgo_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Coffee,Deli / Bodega,Diner,Empanada Restaurant,Fast Food Restaurant,Food Truck,Fried Chicken Joint,Gastropub,Irish Pub,Italian Restaurant,Japanese Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant,Wings Joint
0,Barrio Asta Bandera,0.05,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.05,0.15,0.00,0.00,0.20,0.00,0.10,0.00,0.00,0.00,0.15,0.10,0.05,0.00,0.00
1,Barrio Camelia,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.05,0.05,0.05,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.10,0.15,0.05,0.00,0.00,0.00,0.05,0.00,0.25,0.00,0.00,0.00
2,Colonia 20 de Noviembre,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.15,0.05,0.10,0.00,0.00,0.00,0.00,0.00,0.05,0.25,0.00,0.00,0.00,0.00,0.05,0.00,0.25,0.00,0.00,0.00
3,Colonia Adolfo López Mateos,0.05,0.05,0.00,0.10,0.00,0.10,0.00,0.00,0.05,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.05,0.20,0.00,0.00,0.00,0.00,0.00,0.10,0.15,0.00,0.00,0.00
4,Colonia Ampliación Felipe Ángeles,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.15,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.05,0.00,0.05,0.00,0.00,0.30,0.05,0.00,0.05,0.00,0.00,0.00,0.10,0.05,0.00,0.05
5,Colonia Ampliación San Antonio,0.05,0.00,0.00,0.05,0.05,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.20,0.00,0.15,0.00,0.05,0.05,0.05,0.10,0.00,0.00,0.05
6,Colonia Ampliación Santa Julia,0.05,0.05,0.00,0.05,0.00,0.15,0.00,0.00,0.00,0.05,0.05,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.05,0.00,0.00,0.05,0.15,0.00,0.10,0.00,0.00,0.00,0.10,0.10,0.00,0.00,0.00
7,Colonia Ampliación del Palmar,0.05,0.05,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,0.00,0.05,0.15,0.05,0.00,0.05,0.05,0.00,0.00,0.25,0.00,0.00,0.05
8,Colonia Antonio del Castillo,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.40,0.00,0.00,0.05,0.00,0.00,0.00,0.25,0.00,0.00,0.05
9,Colonia Anáhuac,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.05,0.00,0.05,0.05,0.05,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,0.25,0.05,0.00,0.00,0.00,0.00,0.00,0.15,0.00,0.00,0.05


In [ ]:
hidalgo_grouped.to_csv('hidalgo_grouped.csv')

Clustering for the final data

In [ ]:
# Number of clusters
n_clusters = 5

hidalgo_grouped_clustering = hidalgo_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=n_clusters, random_state=26).fit(hidalgo_grouped_clustering)

kmeans.labels_[0:10]

array([3, 1, 1, 4, 0, 3, 4, 2, 2, 1], dtype=int32)

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hidalgo_grouped['Neighborhood']

for ind in np.arange(hidalgo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hidalgo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barrio Asta Bandera,Restaurant,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,Taco Place
1,Barrio Camelia,Taco Place,Restaurant,Pizza Place,Diner,Deli / Bodega
2,Colonia 20 de Noviembre,Restaurant,Taco Place,Empanada Restaurant,Food Truck,Pizza Place
3,Colonia Adolfo López Mateos,Restaurant,Taco Place,Sushi Restaurant,BBQ Joint,Bakery
4,Colonia Ampliación Felipe Ángeles,Restaurant,Burger Joint,Taco Place,Sandwich Place,Food Truck


In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
hidalgo_merged = data_hidalgo
hidalgo_merged = hidalgo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

hidalgo_merged.head()

,Postal Code,Neighborhood,County,State,County Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,42000,Colonia Centro,Pachuca de Soto,Hidalgo,Urbano,20.1021,-98.7823,3,Restaurant,Taco Place,Sushi Restaurant,Wings Joint,BBQ Joint
1,42015,Barrio Asta Bandera,Pachuca de Soto,Hidalgo,Urbano,20.1021,-98.76713333333332,3,Restaurant,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,Taco Place
2,42027,Colonia Antonio del Castillo,Pachuca de Soto,Hidalgo,Urbano,20.1283,-98.7391,2,Restaurant,Taco Place,Wings Joint,Argentinian Restaurant,Diner
3,42034,Colonia Aquiles Serdán,Pachuca de Soto,Hidalgo,Urbano,20.133,-98.7786,2,Taco Place,Restaurant,BBQ Joint,Sandwich Place,Italian Restaurant
4,42037,Equipamiento Cereso de Pachuca,Pachuca de Soto,Hidalgo,Urbano,20.1229,-98.766,2,Taco Place,Restaurant,Sandwich Place,BBQ Joint,Burger Joint


In [ ]:
hidalgo_merged.to_csv('hidalgo_merged_common.csv')

In [ ]:
#Set some centroid for Folium map

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode('Unidad Medicina Familiar 32, Pachuca, Hidalgo')
latitude = float(location.latitude)
longitude = float(location.longitude)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
c_map = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hidalgo_merged['Latitude'], hidalgo_merged['Longitude'], hidalgo_merged['Neighborhood'], hidalgo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=c_map[cluster-1],
        fill=True,
        fill_color=c_map[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
map_clusters.save("map.html")

In [ ]:
#look for the Clusters

hidalgo_merged.loc[hidalgo_merged['Cluster Labels'] == 4, hidalgo_merged.columns[[1] + list(range(5, hidalgo_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Fraccionamiento 16 de Enero,20.1113,-98.75512857142859,4,Restaurant,Bakery,Taco Place,BBQ Joint,Pizza Place
11,Colonia Boulevares de San Francisco,20.1021,-98.749025,4,Restaurant,Taco Place,Sushi Restaurant,Bakery,American Restaurant
12,Colonia Ampliación Santa Julia,20.1024,-98.76135,4,Restaurant,Bakery,Taco Place,Sushi Restaurant,Seafood Restaurant
19,Fraccionamiento 18 de Marzo,20.0952,-98.75348333333334,4,Restaurant,Bakery,Pizza Place,BBQ Joint,Coffee
20,Colonia Adolfo López Mateos,20.1036,-98.7491,4,Restaurant,Taco Place,Sushi Restaurant,BBQ Joint,Bakery
